In [ ]:
import numpy as np
from lyft_dataset_sdk.lyftdataset import LyftDataset

In [ ]:
import json
import math
import os
import sys
import time
from datetime import datetime
from pathlib import Path
from typing import List, Tuple

import cv2
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
from matplotlib.axes import Axes
from PIL import Image
from pyquaternion import Quaternion
from tqdm import tqdm

from lyft_dataset_sdk.utils.data_classes import Box, LidarPointCloud, RadarPointCloud  # NOQA
from lyft_dataset_sdk.utils.geometry_utils import BoxVisibility, box_in_image, view_points  # NOQA
from lyft_dataset_sdk.utils.map_mask import MapMask

In [ ]:
class gt_box_pointrcnn:
    #     (-y, -z, x, h, w, l, ry) is seven_num
    def __init__(self, seven_num):
        self.center = seven_num[[2, 0, 1]]
        self.center[1] = -self.center[1]
        self.center[2] = -self.center[2]
        self.wlh = seven_num[[4, 5, 3]]
        self.center[2] += self.wlh[2]/2
        self.ry = seven_num[6]

    def corners(self, wlh_factor: float = 1.0) -> np.ndarray:
        """Returns the bounding box corners.

        Args:
            wlh_factor: Multiply width, length, height by a factor to scale the box.

        Returns: First four corners are the ones facing forward.
                The last four are the ones facing backwards.

        """

        width, length, height = self.wlh * wlh_factor

        # 3D bounding box corners. (Convention: x points forward, y to the left, z up.)
        x_corners = length / 2 * np.array([1, 1, 1, 1, -1, -1, -1, -1])
        y_corners = width / 2 * np.array([1, -1, -1, 1, 1, -1, -1, 1])
        z_corners = height / 2 * np.array([1, 1, -1, -1, 1, 1, -1, -1])
        corners = np.vstack((x_corners, y_corners, z_corners))

        # Rotate
        #         corners = np.dot(self.orientation.rotation_matrix, corners)

        # Translate
        x, y, z = self.center
        corners[0, :] = corners[0, :] + x
        corners[1, :] = corners[1, :] + y
        corners[2, :] = corners[2, :] + z
        # TODO np.ndarray 能不能直接合并在一起?不用经过list呢
        return corners

In [ ]:
    def render_rpn_result(pc_input, pred_boxes, seg_result, gt_boxes) -> None:
        """Render 3D visualization of the sample using plotly

        This function is assume that all the necessary components are given and just plot it out to
        varify the outcome of model function
        [pc_input, external_data, seg_result, gt_boxes] are give by evauation function,
        boxes are given by external_data

        Args:
            sample_id: Unique sample identifier.
            render_sample: call level5data.render_sample (Render all LIDAR and camera sample_data in sample along with annotations.)

        """
        import pandas as pd
        import plotly.graph_objects as go

        pc_input = pc_input[:, [2, 0, 1]]
        pc_input[:, 1] = -pc_input[:, 1]
        pc_input[:, 2] = -pc_input[:, 2]
        
        
        bg_flag = np.where(seg_result == 0)[0]
        df_tmp = pd.DataFrame(pc_input[bg_flag], columns=['x', 'y', 'z'])

        # df_tmp['norm'] = np.sqrt(np.power(df_tmp[['x', 'y', 'z']].values, 2).sum(axis=1))

        bg_scatter = go.Scatter3d(
            x=df_tmp['x'],
            y=df_tmp['y'],
            z=df_tmp['z'],
            mode='markers',
            marker=dict(
                size=1,
                color='blue',
                opacity=0.8
            )
        )

        fg_flag = np.where(seg_result == 1)[0]
        df_tmp1 = pd.DataFrame(pc_input[fg_flag], columns=['x', 'y', 'z'])
        fg_scatter = go.Scatter3d(
            x=df_tmp1['x'],
            y=df_tmp1['y'],
            z=df_tmp1['z'],
            mode='markers',
            marker=dict(
                size=1,
                color='yellow',
                opacity=0.8
            )
        )

        x_lines = []
        y_lines = []
        z_lines = []

        def f_lines_add_nones():
            x_lines.append(None)
            y_lines.append(None)
            z_lines.append(None)

        ixs_box_0 = [0, 1, 2, 3, 0]
        ixs_box_1 = [4, 5, 6, 7, 4]

        # Here, they read the boxes and the boxes is alreadey converted to vehicle coordinate
        # But it's KITTI format, so convert it to NuScence

        boxes = [gt_box_pointrcnn(single_box) for single_box in pred_boxes]

        for box in boxes:
            points = view_points(box.corners(), view=np.eye(3), normalize=False)
            x_lines.extend(points[0, ixs_box_0])
            y_lines.extend(points[1, ixs_box_0])
            z_lines.extend(points[2, ixs_box_0])
            f_lines_add_nones()
            x_lines.extend(points[0, ixs_box_1])
            y_lines.extend(points[1, ixs_box_1])
            z_lines.extend(points[2, ixs_box_1])
            f_lines_add_nones()
            for i in range(4):
                x_lines.extend(points[0, [ixs_box_0[i], ixs_box_1[i]]])
                y_lines.extend(points[1, [ixs_box_0[i], ixs_box_1[i]]])
                z_lines.extend(points[2, [ixs_box_0[i], ixs_box_1[i]]])
                f_lines_add_nones()

        lines = go.Scatter3d(
            x=x_lines,
            y=y_lines,
            z=z_lines,
            mode='lines',
            name='lines',
            marker=dict(color='red')
        )

        x_lines = []
        y_lines = []
        z_lines = []
        
        gt_boxes = [gt_box_pointrcnn(single_box) for single_box in gt_boxes]
        for box in gt_boxes:
            points = view_points(box.corners(), view=np.eye(3), normalize=False)
            x_lines.extend(points[0, ixs_box_0])
            y_lines.extend(points[1, ixs_box_0])
            z_lines.extend(points[2, ixs_box_0])
            f_lines_add_nones()
            x_lines.extend(points[0, ixs_box_1])
            y_lines.extend(points[1, ixs_box_1])
            z_lines.extend(points[2, ixs_box_1])
            f_lines_add_nones()
            for i in range(4):
                x_lines.extend(points[0, [ixs_box_0[i], ixs_box_1[i]]])
                y_lines.extend(points[1, [ixs_box_0[i], ixs_box_1[i]]])
                z_lines.extend(points[2, [ixs_box_0[i], ixs_box_1[i]]])
                f_lines_add_nones()

        gt_truth = go.Scatter3d(
            x=x_lines,
            y=y_lines,
            z=z_lines,
            mode='lines',
            name='lines',
            marker=dict(color='green')
        )

        fig = go.Figure(data=[bg_scatter, fg_scatter, lines, gt_truth])
        fig.update_layout(scene_aspectmode='data')
        fig.show()

In [ ]:
path_pre = '/home2/lhr/dataset/full_lyft24'
sample_list = np.sort(os.listdir(path_pre))
# sample_list.sort(key=lambda x: int(x[:2]))
result_pre ='/home2/lhr/cdh_ws/kaggle/PointRCNN/data/debug/rpn'
result_list = np.sort(os.listdir(result_pre))

In [ ]:
# ('cur_pts_rect', type(cur_pts_rect), 'gt_boxes3d', type(gt_boxes3d[bs_idx]))
# debug for rpn result
save_result = np.load(os.path.join(result_pre, result_list[2]), allow_pickle=True)
pc_input, pred_boxes, seg_result, gt_boxes= save_result
# save_result
render_rpn_result(pc_input, pred_boxes, seg_result, gt_boxes)

In [ ]:
def render_rcnn_result(pc_input, pred_boxes, seg_result, gt_boxes, roi_boxes3d_np=None, rpn_cls_np=None) -> None:
    """Render 3D visualization of the sample using plotly

    This file is going to render eval result of PointRCNN's rcnn period result.
    with
                    save_list = [rpn_xyz_np[k], pred_boxes3d_np[k], seg_result_np[k],
                             gt_boxes3d[k], roi_boxes3d_np[k], rpn_cls_np[k]]

    This function is assume that all the necessary components are given and just plot it out to
    varify the outcome of model function
    [pc_input, external_data, seg_result, gt_boxes] are give by evauation function,
    boxes are given by external_data

    Args:
        sample_id: Unique sample identifier.
        render_sample: call level5data.render_sample (Render all LIDAR and camera sample_data in sample along with annotations.)

    """
    import pandas as pd
    import plotly.graph_objects as go

    pc_input = pc_input[:, [2, 0, 1]]
    pc_input[:, 1] = -pc_input[:, 1]
    pc_input[:, 2] = -pc_input[:, 2]

    if rpn_cls_np is None:
        bg_flag = np.where(seg_result == 0)[0]
        df_tmp = pd.DataFrame(pc_input[bg_flag], columns=['x', 'y', 'z'])

        # df_tmp['norm'] = np.sqrt(np.power(df_tmp[['x', 'y', 'z']].values, 2).sum(axis=1))

        bg_scatter = go.Scatter3d(
            x=df_tmp['x'],
            y=df_tmp['y'],
            z=df_tmp['z'],
            mode='markers',
            name='bg_points',
            marker=dict(
                size=1,
                color='blue',
                opacity=0.8
            )
        )

        fg_flag = np.where(seg_result == 1)[0]
        df_tmp1 = pd.DataFrame(pc_input[fg_flag], columns=['x', 'y', 'z'])
        fg_scatter = go.Scatter3d(
            x=df_tmp1['x'],
            y=df_tmp1['y'],
            z=df_tmp1['z'],
            mode='markers',
            name='fg_points',
            marker=dict(
                size=1,
                color='yellow',
                opacity=0.8
            )
        )
    else:
        df_tmp = pd.DataFrame(pc_input, columns=['x', 'y', 'z'])

        # df_tmp['norm'] = np.sqrt(np.power(df_tmp[['x', 'y', 'z']].values, 2).sum(axis=1))

        point_scatter = go.Scatter3d(
            x=df_tmp['x'],
            y=df_tmp['y'],
            z=df_tmp['z'],
            mode='markers',
            name='PointCloud-Class',
            marker=dict(
                size=1,
                color=rpn_cls_np,
                opacity=0.8
            )
        )

    x_lines = []
    y_lines = []
    z_lines = []

    def f_lines_add_nones():
        x_lines.append(None)
        y_lines.append(None)
        z_lines.append(None)

    ixs_box_0 = [0, 1, 2, 3, 0]
    ixs_box_1 = [4, 5, 6, 7, 4]

    # Here, they read the boxes and the boxes is alreadey converted to vehicle coordinate
    # But it's KITTI format, so convert it to NuScence

    boxes = [gt_box_pointrcnn(single_box) for single_box in pred_boxes]

    for box in boxes:
        points = view_points(box.corners(), view=np.eye(3), normalize=False)
        x_lines.extend(points[0, ixs_box_0])
        y_lines.extend(points[1, ixs_box_0])
        z_lines.extend(points[2, ixs_box_0])
        f_lines_add_nones()
        x_lines.extend(points[0, ixs_box_1])
        y_lines.extend(points[1, ixs_box_1])
        z_lines.extend(points[2, ixs_box_1])
        f_lines_add_nones()
        for i in range(4):
            x_lines.extend(points[0, [ixs_box_0[i], ixs_box_1[i]]])
            y_lines.extend(points[1, [ixs_box_0[i], ixs_box_1[i]]])
            z_lines.extend(points[2, [ixs_box_0[i], ixs_box_1[i]]])
            f_lines_add_nones()

    pre_box = go.Scatter3d(
        x=x_lines,
        y=y_lines,
        z=z_lines,
        mode='lines',
        name='pred_boxes',
        marker=dict(color='red')
    )

    x_lines = []
    y_lines = []
    z_lines = []

    gt_boxes = [gt_box_pointrcnn(single_box) for single_box in gt_boxes]
    for box in gt_boxes:
        points = view_points(box.corners(), view=np.eye(3), normalize=False)
        x_lines.extend(points[0, ixs_box_0])
        y_lines.extend(points[1, ixs_box_0])
        z_lines.extend(points[2, ixs_box_0])
        f_lines_add_nones()
        x_lines.extend(points[0, ixs_box_1])
        y_lines.extend(points[1, ixs_box_1])
        z_lines.extend(points[2, ixs_box_1])
        f_lines_add_nones()
        for i in range(4):
            x_lines.extend(points[0, [ixs_box_0[i], ixs_box_1[i]]])
            y_lines.extend(points[1, [ixs_box_0[i], ixs_box_1[i]]])
            z_lines.extend(points[2, [ixs_box_0[i], ixs_box_1[i]]])
            f_lines_add_nones()

    gt_truth = go.Scatter3d(
        x=x_lines,
        y=y_lines,
        z=z_lines,
        mode='lines',
        name='gt_truth',
        marker=dict(color='green')
    )

    if roi_boxes3d_np is not None:

        x_lines = []
        y_lines = []
        z_lines = []

        gt_boxes = [gt_box_pointrcnn(single_box) for single_box in roi_boxes3d_np]
        for box in gt_boxes:
            points = view_points(box.corners(), view=np.eye(3), normalize=False)
            x_lines.extend(points[0, ixs_box_0])
            y_lines.extend(points[1, ixs_box_0])
            z_lines.extend(points[2, ixs_box_0])
            f_lines_add_nones()
            x_lines.extend(points[0, ixs_box_1])
            y_lines.extend(points[1, ixs_box_1])
            z_lines.extend(points[2, ixs_box_1])
            f_lines_add_nones()
            for i in range(4):
                x_lines.extend(points[0, [ixs_box_0[i], ixs_box_1[i]]])
                y_lines.extend(points[1, [ixs_box_0[i], ixs_box_1[i]]])
                z_lines.extend(points[2, [ixs_box_0[i], ixs_box_1[i]]])
                f_lines_add_nones()

        roi_truth = go.Scatter3d(
            x=x_lines,
            y=y_lines,
            z=z_lines,
            mode='lines',
            name='roi_boxes3d',
            marker=dict(color='purple')
        )
    
    if rpn_cls_np is not None:
        data = [point_scatter, pre_box, gt_truth]
    else:
        data=[bg_scatter, fg_scatter, pre_box, gt_truth]
    if roi_boxes3d_np is not None:
        data.append(roi_truth)
        
    fig = go.Figure(data=data)
    fig.update_layout(scene_aspectmode='data')
    fig.show()

In [ ]:
path_pre = '/home2/lhr/dataset/full_lyft24'
sample_list = np.sort(os.listdir(path_pre))
# sample_list.sort(key=lambda x: int(x[:2]))
result_pre ='/home2/lhr/cdh_ws/kaggle/PointRCNN/data/debug/rcnn'
result_list = np.sort(os.listdir(result_pre))

In [ ]:
path_pre = '/home2/lhr/dataset/full_lyft24'
sample_list = np.sort(os.listdir(path_pre))
# sample_list.sort(key=lambda x: int(x[:2]))
result_pre ='/home2/lhr/cdh_ws/kaggle/PointRCNN/data/debug/rcnn'
result_list = np.sort(os.listdir(result_pre))

In [ ]:
# ('cur_pts_rect', type(cur_pts_rect), 'gt_boxes3d', type(gt_boxes3d[bs_idx]))
# debug for rpn result
a = 0
for i in range(a, a+3):
    save_result = np.load(os.path.join(result_pre, result_list[i]), allow_pickle=True)
    pc_input, pred_boxes, seg_result, gt_boxes, roi_boxes3d_np, rpn_cls_np = save_result
    # save_result
    render_rcnn_result(pc_input, pred_boxes, seg_result, gt_boxes, roi_boxes3d_np=roi_boxes3d_np, rpn_cls_np=None)

In [ ]:
np.sqrt(np.power(pc_input[:, [0, 2]], 2)).shape

In [ ]:
np.sum(np.power(pc_input[:, [0, 2]], 2), axis=1).shape

In [ ]:
level5data = LyftDataset(data_path='/home2/lhr/dataset/level5dataset/v1.02-train',
                         json_path='/home2/lhr/dataset/level5dataset/v1.02-train/v1.02-train', verbose=True)

In [ ]:
self = level5data